In [6]:
from astropy.io import fits
import numpy as np

# Load ASCII data
data = np.genfromtxt(r'C:\Users\khush\Downloads\Cross_section\scattered_spectrum1.txt', delimiter='\t', dtype=None, encoding='utf-8', names=True)

# Extract data columns
energy_keV = data['Energy_keV']
scattered_photons = data['Scattered_Photons']

# Define lower and upper bounds for energy bins
energy_lo = energy_keV[:-1]  # Lower bounds of each energy bin
energy_hi = energy_keV[1:]   # Upper bounds of each energy bin

# Primary HDU
primary_hdu = fits.PrimaryHDU()
primary_hdu.header['MODLNAME'] = 'scatter_model'
primary_hdu.header['MODLUNIT'] = 'photons/(cm^2 s)'
primary_hdu.header['ADDMODEL'] = True
primary_hdu.header['HDUCLASS'] = 'OGIP'
primary_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
primary_hdu.header['HDUVERS'] = '1.0.0'

# PARAMETERS extension
param_data = np.array([
    ('scaling_factor', 0, 1.0, 0.1, 0.0, 0.0, 10.0, 10.0, 1, [1.0])
], dtype=[
    ('NAME', 'S12'), ('METHOD', 'i4'), ('INITIAL', 'f4'), ('DELTA', 'f4'),
    ('MINIMUM', 'f4'), ('BOTTOM', 'f4'), ('TOP', 'f4'), ('MAXIMUM', 'f4'),
    ('NUMBVALS', 'i4'), ('VALUE', 'f4', (1,))
])

param_cols = fits.ColDefs([
    fits.Column(name='NAME', format='12A', array=param_data['NAME']),
    fits.Column(name='METHOD', format='J', array=param_data['METHOD']),
    fits.Column(name='INITIAL', format='E', array=param_data['INITIAL']),
    fits.Column(name='DELTA', format='E', array=param_data['DELTA']),
    fits.Column(name='MINIMUM', format='E', array=param_data['MINIMUM']),
    fits.Column(name='BOTTOM', format='E', array=param_data['BOTTOM']),
    fits.Column(name='TOP', format='E', array=param_data['TOP']),
    fits.Column(name='MAXIMUM', format='E', array=param_data['MAXIMUM']),
    fits.Column(name='NUMBVALS', format='J', array=param_data['NUMBVALS']),
    fits.Column(name='VALUE', format='E', array=param_data['VALUE'])
])
param_hdu = fits.BinTableHDU.from_columns(param_cols)
param_hdu.header['HDUCLASS'] = 'OGIP'
param_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
param_hdu.header['HDUCLAS2'] = 'PARAMETERS'
param_hdu.header['HDUVERS'] = '1.0.0'
param_hdu.header['EXTNAME'] = 'PARAMETERS'  # Set extension name

# ENERGIES extension
energy_cols = fits.ColDefs([
    fits.Column(name='ENERG_LO', format='E', array=energy_lo),
    fits.Column(name='ENERG_HI', format='E', array=energy_hi)
])
energy_hdu = fits.BinTableHDU.from_columns(energy_cols)
energy_hdu.header['HDUCLASS'] = 'OGIP'
energy_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
energy_hdu.header['HDUCLAS2'] = 'ENERGIES'
energy_hdu.header['HDUVERS'] = '1.0.0'
energy_hdu.header['EXTNAME'] = 'ENERGIES'  # Set extension name

# SPECTRA extension with scaling applied
paramval = np.arange(0.0, 1.5, 0.1)
num_bins = 3000
scaling_factor = 10000000000000.1640448511589367
intpspec = np.tile(scattered_photons[:num_bins], (len(paramval), 1)) * scaling_factor  # Apply scaling factor

spectrum_cols = fits.ColDefs([
    fits.Column(name='PARAMVAL', format='E', array=paramval),
    fits.Column(name='INTPSPEC', format=f'{num_bins}E', array=intpspec)
])
spectrum_hdu = fits.BinTableHDU.from_columns(spectrum_cols)
spectrum_hdu.header['HDUCLASS'] = 'OGIP'
spectrum_hdu.header['HDUCLAS1'] = 'XSPEC TABLE MODEL'
spectrum_hdu.header['HDUCLAS2'] = 'MODEL SPECTRA'
spectrum_hdu.header['HDUVERS'] = '1.0.0'
spectrum_hdu.header['EXTNAME'] = 'SPECTRA'  # Set extension name

# Create HDU list and save to FITS file
output_path = r'C:/Users/khush/Downloads/Cross_section/scattered_solar_spectrum_model_v17.fits'
hdul = fits.HDUList([primary_hdu, param_hdu, energy_hdu, spectrum_hdu])
hdul.writeto(output_path, overwrite=True)

print(f"FITS file created successfully at '{output_path}'.")


FITS file created successfully at 'C:/Users/khush/Downloads/Cross_section/scattered_solar_spectrum_model_v17.fits'.
